In [71]:
import pandas as pd

s2015 = pd.read_csv('national_league_2015_2016.csv', parse_dates = ['date']).iloc[5:].reset_index(drop=True)
s2016 = pd.read_csv('national_league_2016_2017.csv', parse_dates = ['date']).iloc[5:].reset_index(drop=True)
s2017 = pd.read_csv('national_league_2017_2018.csv', parse_dates = ['date']).iloc[5:].reset_index(drop=True)

In [72]:
def extract_results(season):
    df = pd.read_csv(f'national_league_{season}_{season+1}.csv', parse_dates = ['date']).iloc[5:]

    ssn_records = []

    games = df.to_dict('records')

    for game in games:
        home_rec = {
            'season': season,
            'date': game['date'],
            'team': game['home_team'],
            'opponent': game['away_team'],
            'venue': 'H',
            'goals_for': game['home_score'],
            'goals_against': game['away_score']
        }

        away_rec = {
            'season': season,
            'date': game['date'],
            'team': game['away_team'],
            'opponent': game['home_team'],
            'venue': 'A',
            'goals_for': game['away_score'],
            'goals_against': game['home_score']
        }
        ssn_records.append(home_rec)
        ssn_records.append(away_rec)
    return ssn_records

In [73]:
all_seasons = []
for season in range(2015, 2018):
    records = extract_results(season)
    all_seasons.extend(records)

In [74]:
fix = pd.DataFrame(all_seasons)
fix = fix.query('team == "Tranmere"')[['season', 'date']]
fix["prev_day"] = fix["date"] - pd.Timedelta(days=1)
fix = fix.to_dict('records')

In [75]:
df = pd.DataFrame(all_seasons)

In [76]:
df["outcome"] = df.apply(lambda row: "W" if row["goals_for"] > row["goals_against"] else "L" if row["goals_for"] < row["goals_against"] else "D", axis=1)

In [77]:
df["points"] = df.apply(lambda row: 3 if row["outcome"] == "W" else 1 if row["outcome"] == "D" else 0, axis=1)

In [78]:
def make_tab(ssn, date):
    grouped = df.query("season == @ssn & date < @date").groupby("team")
    played = grouped.size()
    won = grouped.apply(lambda x: (x["outcome"] == "W").sum(), include_groups=False)
    drawn = grouped.apply(lambda x: (x["outcome"] == "D").sum(), include_groups=False)
    lost = grouped.apply(lambda x: (x["outcome"] == "L").sum(), include_groups=False)
    goals_for = grouped["goals_for"].sum()
    goals_against = grouped["goals_against"].sum()
    goal_diff = goals_for - goals_against
    points = grouped["points"].sum()

    summary = pd.DataFrame({
        "played": played,
        "won": won,
        "drawn": drawn,
        "lost": lost,
        "goals_for": goals_for,
        "goals_against": goals_against,
        "goal_diff": goal_diff,
        "points": points,
        "season": ssn,
        "pregame_date": date
    })

    return summary.sort_values(by=["points", "goal_diff", "goals_for"], ascending=False).reset_index(drop=False)

In [79]:
all_tabs = []

for f in fix:
    ssn = f['season']
    date = f['prev_day']
    try:
        tab = make_tab(ssn, date)
        all_tabs.append(tab)
    except:
        print(f"Season {ssn} up to {date} failed")

Season 2015 up to 2015-08-07 00:00:00 failed
Season 2016 up to 2016-08-05 00:00:00 failed
Season 2017 up to 2017-08-04 00:00:00 failed


In [80]:
all_tabs_df = pd.concat(all_tabs)
all_tabs_df

,team,played,won,drawn,lost,goals_for,goals_against,goal_diff,points,season,pregame_date
0,Cheltenham,45,29,11,5,84,29,55,98,2015,2016-04-29
1,Forest Green,45,25,11,9,68,42,26,86,2015,2016-04-29
2,Grimsby,45,22,14,9,82,44,38,80,2015,2016-04-29
3,Dover,45,23,11,11,75,52,23,80,2015,2016-04-29
4,Braintree,45,22,12,11,53,38,15,78,2015,2016-04-29
...,...,...,...,...,...,...,...,...,...,...,...
17,Gateshead,1,0,0,1,1,2,-1,0,2017,2017-08-07
18,Hartlepool,1,0,0,1,0,1,-1,0,2017,2017-08-07
19,Wrexham,1,0,0,1,0,1,-1,0,2017,2017-08-07
20,FC Halifax,1,0,0,1,0,2,-2,0,2017,2017-08-07


In [81]:
missing = pd.DataFrame([
    {
        "team": "Chester",
        "played": 0,
        "won": 0,
        "drawn": 0,
        "lost": 0,
        "goals_for": 0,
        "goals_against": 0,
        "goal_diff": 0,
        "points": 0,
        "season": 2017,
        "pregame_date": pd.Timestamp("2017-08-07")
    },
    {
        "team": "Solihull Moors",
        "played": 0,
        "won": 0,
        "drawn": 0,
        "lost": 0,
        "goals_for": 0,
        "goals_against": 0,
        "goal_diff": 0,
        "points": 0,
        "season": 2017,
        "pregame_date": pd.Timestamp("2017-08-07")
    }
])

all_tabs_df = pd.concat([all_tabs_df, missing])
all_tabs_df = all_tabs_df.sort_values(by=["season", "pregame_date", "points", "goal_diff", "goals_for"], ascending=False).reset_index(drop=True)

In [82]:
all_tabs_df.to_csv('national_league_tabs.csv', index=False)

In [83]:
all_tabs_df["pos"] = all_tabs_df.groupby("pregame_date").cumcount() + 1
all_tabs_df

,team,played,won,drawn,lost,goals_for,goals_against,goal_diff,points,season,pregame_date,pos
0,Macclesfield,45,26,11,8,65,46,19,89,2017,2018-04-27,1
1,Tranmere,45,24,10,11,77,44,33,82,2017,2018-04-27,2
2,Sutton,45,22,10,13,65,52,13,76,2017,2018-04-27,3
3,Aldershot,45,20,15,10,63,50,13,75,2017,2018-04-27,4
4,Ebbsfleet,45,19,16,10,63,49,14,73,2017,2018-04-27,5
...,...,...,...,...,...,...,...,...,...,...,...,...
3235,Macclesfield,1,0,0,1,0,1,-1,0,2015,2015-08-10,20
3236,Woking,1,0,0,1,0,1,-1,0,2015,2015-08-10,21
3237,FC Halifax,1,0,0,1,1,3,-2,0,2015,2015-08-10,22
3238,Wrexham,1,0,0,1,1,3,-2,0,2015,2015-08-10,23


In [84]:
all_tabs_df["game_date"] = all_tabs_df["pregame_date"] + pd.Timedelta(days=1)

In [85]:
all_tabs_df = all_tabs_df.rename(
    columns={
        "team": "Team",
        "played": "Pld",
        "won": "W",
        "drawn": "D",
        "lost": "L",
        "goals_for": "GF",
        "goals_against": "GA",
        "goal_diff": "GD",
        "points": "Pts"
    }
)[["pos","Team","Pld","W","D","L","GF","GA","GD","Pts","pregame_date","game_date"]]

In [86]:
res = pd.read_csv("https://raw.githubusercontent.com/petebrown/data-updater/main/data/results.csv", parse_dates=["game_date"])

res = res.query("competition == 'National League'")

In [87]:
res = res[["game_date", "ssn_comp_game_no"]].rename(columns={"ssn_comp_game_no": "game_no"})

res

,game_date,game_no
4430,2015-08-08,1
4431,2015-08-11,2
4432,2015-08-15,3
4433,2015-08-18,4
4434,2015-08-22,5
...,...,...
4581,2018-04-14,42
4582,2018-04-17,43
4583,2018-04-21,44
4584,2018-04-24,45


In [88]:
all_tabs_df = all_tabs_df.merge(res, on="game_date", how="left")
all_tabs_df.to_csv('national_league_tabs.csv', index=False)